In [1]:
import math
import numpy as np

import torch
import torchvision
import torchvision.transforms as transform
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt

In [2]:
from K_Transformer import Transformer

### Dataset to check if model running

In [3]:
english_file = r'english.txt' # replace this path with appropriate one
kannada_file = r'kannada.txt' # replace this path with appropriate one

# Generated this by filtering Appendix code

START_TOKEN = '<START>'
PADDING_TOKEN = '<PADDING>'
END_TOKEN = '<END>'

kannada_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', 
                      '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '<', '=', '>', '?', 'ˌ', 
                      'ँ', 'ఆ', 'ఇ', 'ా', 'ి', 'ీ', 'ు', 'ూ', 
                      'ಅ', 'ಆ', 'ಇ', 'ಈ', 'ಉ', 'ಊ', 'ಋ', 'ೠ', 'ಌ', 'ಎ', 'ಏ', 'ಐ', 'ಒ', 'ಓ', 'ಔ', 
                      'ಕ', 'ಖ', 'ಗ', 'ಘ', 'ಙ', 
                      'ಚ', 'ಛ', 'ಜ', 'ಝ', 'ಞ', 
                      'ಟ', 'ಠ', 'ಡ', 'ಢ', 'ಣ', 
                      'ತ', 'ಥ', 'ದ', 'ಧ', 'ನ', 
                      'ಪ', 'ಫ', 'ಬ', 'ಭ', 'ಮ', 
                      'ಯ', 'ರ', 'ಱ', 'ಲ', 'ಳ', 'ವ', 'ಶ', 'ಷ', 'ಸ', 'ಹ', 
                      '಼', 'ಽ', 'ಾ', 'ಿ', 'ೀ', 'ು', 'ೂ', 'ೃ', 'ೄ', 'ೆ', 'ೇ', 'ೈ', 'ೊ', 'ೋ', 'ೌ', '್', 'ೕ', 'ೖ', 'ೞ', 'ೣ', 'ಂ', 'ಃ', 
                      '೦', '೧', '೨', '೩', '೪', '೫', '೬', '೭', '೮', '೯', PADDING_TOKEN, END_TOKEN]

english_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', 
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                        ':', '<', '=', '>', '?', '@',
                        '[', '\\', ']', '^', '_', '`', 
                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                        'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 
                        'y', 'z', 
                        '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]

In [4]:
index_to_kannada = {k:v for k,v in enumerate(kannada_vocabulary)}
kannada_to_index = {v:k for k,v in enumerate(kannada_vocabulary)}
index_to_english = {k:v for k,v in enumerate(english_vocabulary)}
english_to_index = {v:k for k,v in enumerate(english_vocabulary)}

In [17]:
with open(english_file, 'r') as file:
    english_sentences = file.readlines()
with open(kannada_file, 'r', encoding = 'utf8') as file:
    kannada_sentences = file.readlines()

# Limit Number of sentences
TOTAL_SENTENCES = 200000
english_sentences = english_sentences[:TOTAL_SENTENCES]
kannada_sentences = kannada_sentences[:TOTAL_SENTENCES]
english_sentences = [sentence.rstrip('\n').lower() for sentence in english_sentences]
kannada_sentences = [sentence.rstrip('\n') for sentence in kannada_sentences]

In [18]:
k = []
j = []
for i in range(len(english_sentences)):
    if('\xad' in english_sentences[i]):
        k.append(english_sentences[i])
        j.append(kannada_sentences[i])

for i in k:
    english_sentences.remove(i)
for i in j:
    kannada_sentences.remove(i)

print(k)

[]


In [19]:
max_sequence_length = 200

def is_valid_tokens(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True

def is_valid_length(sentence, max_sequence_length):
    return len(list(sentence)) < (max_sequence_length - 1) # need to re-add the end token so leaving 1 space

valid_sentence_indicies = []
for index in range(len(kannada_sentences)):
    kannada_sentence, english_sentence = kannada_sentences[index], english_sentences[index]
    if is_valid_length(kannada_sentence, max_sequence_length) \
      and is_valid_length(english_sentence, max_sequence_length) \
      and is_valid_tokens(kannada_sentence, kannada_vocabulary):
        valid_sentence_indicies.append(index)

print(f"Number of sentences: {len(kannada_sentences)}")
print(f"Number of valid sentences: {len(valid_sentence_indicies)}")

Number of sentences: 200000
Number of valid sentences: 164022


In [20]:
kannada_sentences = [kannada_sentences[i] for i in valid_sentence_indicies]
english_sentences = [english_sentences[i] for i in valid_sentence_indicies]

In [21]:
d_model = 512
batch_size = 30
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.1
num_layers = 1
max_sequence_length = 500
kn_vocab_size = len(kannada_vocabulary)

transformer = Transformer(d_model, 
                          ffn_hidden,
                          num_heads, 
                          drop_prob, 
                          num_layers, 
                          max_sequence_length,
                          kn_vocab_size,
                          english_to_index,
                          kannada_to_index,
                          START_TOKEN, 
                          END_TOKEN, 
                          PADDING_TOKEN)

In [22]:
transformer

Transformer(
  (encoder): Encoder(
    (sentence_embedding): SentenceEmbedding(
      (embedding): Embedding(71, 512)
      (position_encoder): PositionalEncoding()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): SequentialEncoder(
      (0): EncoderLayer(
        (attention): MultiHeadAttention(
          (qkv_layer): Linear(in_features=512, out_features=1536, bias=True)
          (linear_layer): Linear(in_features=512, out_features=512, bias=True)
        )
        (norm1): LayerNormalization()
        (dropout1): Dropout(p=0.1, inplace=False)
        (ffn): PositionwiseFeedForward(
          (linear1): KANLinear(
            (base_activation): SiLU()
          )
          (linear2): KANLinear(
            (base_activation): SiLU()
          )
          (relu): ReLU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (norm2): LayerNormalization()
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Decoder(
    (senten

In [23]:
class TextDataset(Dataset):

    def __init__(self, english_sentences, kannada_sentences):
        self.english_sentences = english_sentences
        self.kannada_sentences = kannada_sentences

    def __len__(self):
        return len(self.english_sentences)

    def __getitem__(self, idx):
        return self.english_sentences[idx], self.kannada_sentences[idx]

In [24]:
dataset = TextDataset(english_sentences, kannada_sentences)

In [25]:
train_loader = DataLoader(dataset, batch_size)
iterator = iter(train_loader)

In [26]:
criterian = nn.CrossEntropyLoss(ignore_index=kannada_to_index[PADDING_TOKEN],
                                reduction='none')

# When computing the loss, we are ignoring cases when the label is the padding token
for params in transformer.parameters():
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)

optim = torch.optim.Adam(transformer.parameters(), lr=1e-4)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [27]:
NEG_INFTY = -1e9

def create_masks(eng_batch, kn_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length] , True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)

    for idx in range(num_sentences):
      eng_sentence_length, kn_sentence_length = len(eng_batch[idx]), len(kn_batch[idx])
      eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_sequence_length)
      kn_chars_to_padding_mask = np.arange(kn_sentence_length + 1, max_sequence_length)
      encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
      encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
      decoder_padding_mask_self_attention[idx, :, kn_chars_to_padding_mask] = True
      decoder_padding_mask_self_attention[idx, kn_chars_to_padding_mask, :] = True
      decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True
      decoder_padding_mask_cross_attention[idx, kn_chars_to_padding_mask, :] = True

    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0.0)
    decoder_self_attention_mask =  torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0.0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0.0)
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask

In [28]:
transformer.train()
transformer.to(device)
total_loss = 0
num_epochs = 10

for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    iterator = iter(train_loader)
    for batch_num, batch in enumerate(iterator):
        transformer.train()
        eng_batch, kn_batch = batch
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(eng_batch, kn_batch)
        optim.zero_grad()
        kn_predictions = transformer(eng_batch,
                                     kn_batch,
                                     encoder_self_attention_mask.to(device), 
                                     decoder_self_attention_mask.to(device), 
                                     decoder_cross_attention_mask.to(device),
                                     enc_start_token=False,
                                     enc_end_token=False,
                                     dec_start_token=True,
                                     dec_end_token=True)
        labels = transformer.decoder.sentence_embedding.batch_tokenize(kn_batch, start_token=False, end_token=True)
        loss = criterian(
            kn_predictions.view(-1, kn_vocab_size).to(device),
            labels.view(-1).to(device)
        ).to(device)
        valid_indicies = torch.where(labels.view(-1) == kannada_to_index[PADDING_TOKEN], False, True)
        loss = loss.sum() / valid_indicies.sum()
        loss.backward()
        optim.step()
        #train_losses.append(loss.item())
        if batch_num % 100 == 0:
            print(f"Iteration {batch_num} : {loss.item()}")
            print(f"English: {eng_batch[0]}")
            print(f"kannada Translation: {kn_batch[0]}")
            kn_sentence_predicted = torch.argmax(kn_predictions[0], axis=1)
            predicted_sentence = ""
            for idx in kn_sentence_predicted:
              if idx == kannada_to_index[END_TOKEN]:
                break
              predicted_sentence += index_to_kannada[idx.item()]
            print(f"kannada Prediction: {predicted_sentence}")


            transformer.eval()
            kn_sentence = ("",)
            eng_sentence = ("should we go to the mall?",)
            for word_counter in range(max_sequence_length):
                encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, kn_sentence)
                predictions = transformer(eng_sentence,
                                          kn_sentence,
                                          encoder_self_attention_mask.to(device), 
                                          decoder_self_attention_mask.to(device), 
                                          decoder_cross_attention_mask.to(device),
                                          enc_start_token=False,
                                          enc_end_token=False,
                                          dec_start_token=True,
                                          dec_end_token=False)
                next_token_prob_distribution = predictions[0][word_counter] # not actual probs
                next_token_index = torch.argmax(next_token_prob_distribution).item()
                next_token = index_to_kannada[next_token_index]
                kn_sentence = (kn_sentence[0] + next_token, )
                if next_token == END_TOKEN:
                  break
            
            print(f"Evaluation translation (should we go to the mall?) : {kn_sentence}")
            print("-------------------------------------------")

Epoch 0
Iteration 0 : 6.033585071563721
English: hes a scientist.
kannada Translation: ಇವರು ಸಂಶೋಧಕ ಸ್ವಭಾವದವರು.
kannada Prediction: ా4ು44444444444444ಖ444ಥಥ4ಭಭಓ+ೕ+ಭಭಭಭಭಭಭಭಭಭಭಭಭಭಭ'ತಭಭಭಭಭಭಭಭೃಭ8ಚ+ತಭತತತತ8ಃ೮ಭತತತತತ+++ತತತಓತಭಐಭತತತತತತತತತತಭತೊತ+ತಭಭಭ::::ೃ:ೃ೨ೊತಃೊ88ಓ8ಚ8ಚಚಚಚಥ8888ತಚಧ88888ಚಚಚಞೊಞಞಚಃ8ಚಚಚಚ88ಭಃಃ8<ೞ<<<<ೃತಭಭತತಃಃತಭಭಭ೭<ಭಭಓಃ0ಃಱ8ಭಱಭ<<<ಭ<ಱಱ<ೃಱಭ<ತೃ+ಃಃ+ా+ಃಃಃಃಐಐಐಐಱಱಃ8ಐ8ಐಃಱ+ಭ+<ೣತಚಒ8ಞಞಭಱಱ.ಱಱಱ+ೄಭಭ8ಓೄ8ಚೄ88+8ಠಭಠಠಠಱಠಠಠಠಠಱ+ಓ8ಠಓಓಓಃಃಓಃ8ಃ8ೈಱಃ8888ಃಃಃಒಱಱಱಠಱಃಃಃ.ಃಃಃಃಃಐೞಃೞೞಃಃ೭ಃಃಃಃಉಃಃ೭ಃಃಃಃಃಃಃಃಃಃಃಃಃಃಃಃೞಠಠ೭ಠಃೌಱೌಃಃಧಃಃಱೃೃ9#ೈೈೈೈಠ#ಱಱಱ0ಉಉೌೕೌೕಱೃೕೃೕೃಱೕಱಱೕೕೀೕಱೕ<<<೦/ಃಃೞಃೞೄಃಃಃಃೈೈಚೈೈಉೈೈೈೈೈಔಔೈೈೈೈಭೈೈಃಃఆಃಃಃಃ++ಃ+ಃಃಃಃಃಃಃಃಃಃాಃಃಃಃಃಃ<=ೈೈೈಃಃಃಃಓ೫&8ಓ+++8ೕಚೈೈ=ೈ5ಃೈೈಃಃಃಃಃೞಃಃೞಔೞೞಃೞೞ
Evaluation translation (should we go to the mall?) : ('ುುುಪಪುುುುುುಧಧಖಖಖಖಖಖಧಧಧಥಥಥాಥ99ಥಥಥ4ೃಭಭಭಭಭಧಧಧಭಭೃೃತತತಭಭಭೃೃೃೃೃತಚಚಚಚತತತತತತತತತತತತತತತತುತತತಐಐಐಐಐಐಐತತತತೊೊಐಐಐಐಐಐಐಐಐಐಐ::ಐಐಐಐಐಐಐಐಐಐಐಐಐಐಐಐಚಐಚಥೊೊೊಐಚಚಚ   ಚಚಚಚಚ    ಚ ಚಚಚಚಚಚಚ ಪಪಚಚಚಚಚೃೃೃೃಐాತತಚೃಐಐ9999ಐಐಐಐಐಐಐಐೃೃೃಐಐಐಐೃೃೃೃೃೃೃೃೃೃೃಃಃಃಚಐಐಐಐಐಐಐಐಐಐಐಐಐಐಐಐಐಐೃೃಐಐಐಐಪಪಪೊೊೊಪಪಪಪಪಪಪಪಐಐಐಠಠಚೃೃಐಐಐಪಪಠಠಠಠಠಠಠಠಠಠಠಠಪಪಪಪಐಐಐಐಐಃಐಐಚಚಚಚಚಚಪಪಪಪಠಚೃೃಠಠಠಠಠಠಠಠಪಪಪಐಐಐ

In [30]:
transformer.eval()
def translate(eng_sentence):
  eng_sentence = (eng_sentence,)
  kn_sentence = ("",)
  for word_counter in range(max_sequence_length):
    encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, kn_sentence)
    predictions = transformer(eng_sentence,
                              kn_sentence,
                              encoder_self_attention_mask.to(device), 
                              decoder_self_attention_mask.to(device), 
                              decoder_cross_attention_mask.to(device),
                              enc_start_token=False,
                              enc_end_token=False,
                              dec_start_token=True,
                              dec_end_token=False)
    next_token_prob_distribution = predictions[0][word_counter]
    next_token_index = torch.argmax(next_token_prob_distribution).item()
    next_token = index_to_kannada[next_token_index]
    kn_sentence = (kn_sentence[0] + next_token, )
    if next_token == END_TOKEN:
      break
  return kn_sentence[0]

In [31]:
translation = translate("what should we do when the day starts?")
print(translation)
#ದಿನ ಪ್ರಾರಂಭವಾದಾಗ ನಾವು ಏನು ಮಾಡಬೇಕು?

#Translation given below means: What should we do?

ನಾವು ಏನು ಮಾಡಬೇಕು?<END>


In [32]:
translation = translate("how is this the truth?")
print(translation)
#ಇದು ಹೇಗೆ ಸತ್ಯ

#Translation given below: How is this possible?

ಇದು ಹೇಗೆ ಸಾಧ್ಯವಿದೆ?<END>


In [33]:
translation = translate("the world is a large place with different people")
print(translation)
#ಪ್ರಪಂಚವು ವಿಭಿನ್ನ ಜನರೊಂದಿಗೆ ದೊಡ್ಡ ಸ್ಥಳವಾಗಿದೆ

#From this, the local sold in the local market

ಇದರಿಂದ ಸ್ಥಳೀಯ ಮಾರುಕಟ್ಟೆಯಲ್ಲಿ ಸ್ಥಳೀಯ ಮಾರಾಟ ಮಾಡಿದ್ದಾರೆ<END>


In [34]:
translation = translate("i cannot stand this smell")
print(translation)
#ನಾನು ಈ ವಾಸನೆಯನ್ನು ಸಹಿಸುವುದಿಲ್ಲ

#I can't do it

ನಾನು ಮಾಡುವ ಸಾಧ್ಯವಿಲ್ಲ<END>


In [35]:
translation = translate("noodles are the best")
print(translation)

#No death here

ಇಲ್ಲಿ ಯಾವುದೇ ಸಾವು<END>


In [36]:
translation = translate("why care about this?")
print(translation)

#Why what in the context of the relationship?

ಏಕೆ ಸಂಬಂಧ ಸಂದರ್ಭದಲ್ಲಿ ಏನು?<END>


In [37]:
translation = translate("this is the best thing ever")
print(translation)
# ಇದು ಎಂದೆಂದಿಗೂ ಉತ್ತಮವಾಗಿದೆ

#It is in quite a place

ಇದು ಸಾಕಷ್ಟು ಸ್ಥಳದಲ್ಲಿ ಇದೆ<END>


In [38]:
translation = translate("i am here")
print(translation)
# ನಾನು ಇಲ್ಲಿದ್ದೇನೆ

#I have seen

ನಾನು ನಾನು ನಾನು ನಾನು ನೋಡಿದ್ದೇನೆ.<END>


In [39]:
translation = translate("what should we do?")
print(translation)

#What should we do?

ನಾವು ಏನು ಮಾಡಬೇಕು?<END>


In [40]:
translation = translate("today, what should we do")
print(translation)

#They have done and done

ಅವರು ಮಾಡಿದ ಮಾಡಿದ ಮಾಡಿದ್ದಾರೆ<END>


In [41]:
translation = translate("why did they do this?")
print(translation)
# ಅವರು ಇದನ್ನು ಏಕೆ ಮಾಡಿದರು?

#Why did they say what they did?

ಅವರು ಏಕೆ ಮಾಡಿದ್ದು ಏಕೆ ಹೇಳಿದ್ದಾರೆ?<END>


In [42]:
translation = translate("i am well.")
print(translation)
# ನಾನು ಆರಾಮವಾಗಿದ್ದೇನೆ

#I have seen

ನಾನು ನಾನು ನಾನು ನಾನು ನೋಡಿದ್ದೇನೆ.<END>


In [43]:
translation = translate("whats the word on the street?")
print(translation)

#What to do about it?

ಇದರ ಬಗ್ಗೆ ಏನು ಮಾಡಬೇಕು?<END>
